In [1]:
import os
import random
import pandas as pd
import numpy as np
import warnings
from itertools import zip_longest

warnings.filterwarnings(action='ignore')
from tqdm import tqdm as tq

import torch
from torch.utils.data import Dataset, DataLoader

from transformers import AutoModel, AutoTokenizer



# 임시 토크나이저
from konlpy.tag import Okt, Komoran

## check train data

In [2]:
folder = os.getcwd() + '\\open'
train = os.listdir(folder)[2]
test  = os.listdir(folder)[1]
submit = os.listdir(folder)[0]

train = pd.read_csv(folder + '/' + train)
test = pd.read_csv(folder + '/' + test)
submit = pd.read_csv(folder + '/' + submit)
train.head(5)

,ID,문장,유형,극성,시제,확실성,label
0,TRAIN_00000,0.75%포인트 금리 인상은 1994년 이후 28년 만에 처음이다.,사실형,긍정,현재,확실,사실형-긍정-현재-확실
1,TRAIN_00001,이어 ＂앞으로 전문가들과 함께 4주 단위로 상황을 재평가할 예정＂이라며 ＂그 이전이...,사실형,긍정,과거,확실,사실형-긍정-과거-확실
2,TRAIN_00002,정부가 고유가 대응을 위해 7월부터 연말까지 유류세 인하 폭을 30%에서 37%까지...,사실형,긍정,미래,확실,사실형-긍정-미래-확실
3,TRAIN_00003,"서울시는 올해 3월 즉시 견인 유예시간 60분을 제공하겠다고 밝혔지만, 하루 만에 ...",사실형,긍정,과거,확실,사실형-긍정-과거-확실
4,TRAIN_00004,익사한 자는 사다리에 태워 거꾸로 놓고 소금으로 코를 막아 가득 채운다.,사실형,긍정,현재,확실,사실형-긍정-현재-확실


In [3]:
type1 = train['유형']=='사실형'
type2 = train['유형']=='추론형'
type3 = train['유형']=='예측형'
type4 = train['유형']=='대화형'

sent1 = train['극성']=='긍정'
sent2 = train['극성']=='부정'
sent3 = train['극성']=='미정'

tense1 = train['시제']=='현재'
tense2 = train['시제']=='과거'
tense3 = train['시제']=='미래'

certainty1 = train['확실성']=='확실'
certainty2 = train['확실성']=='불확실'

y1 = np.where(type1, 0, 
          np.where(type2, 1, 
          np.where(type3, 2, 3))).tolist()
y2 = np.where(sent1, 0,
          np.where(sent2, 1, 2)).tolist()
y3 = np.where(tense1, 0, 
          np.where(tense2, 1, 2)).tolist()
y4 = np.where(certainty1, 0, 1).tolist()

y = pd.DataFrame([y1, y2, y3, y4]).T

In [4]:
# 유형, 극성, 시제, 확실성
y.corr()

,0,1,2,3
0,1.000000,0.160107,-0.000258,0.276760
1,0.160107,1.000000,0.036610,0.189302
2,-0.000258,0.036610,1.000000,0.216086
3,0.276760,0.189302,0.216086,1.000000


In [5]:
print(f"1. num train : {len(train)}\n")
print(f"2. null train check : \n{train.isnull().sum()}\n")
print(f"3. unique labels : \n{train.유형.unique()}\n\
{[len(train[train['유형']==train.유형.unique()[idx]]) for idx in range(len(train.유형.unique()))]}\n\
{train.극성.unique()}\n\
{[len(train[train['극성']==train.극성.unique()[idx]]) for idx in range(len(train.극성.unique()))]}\n\
{train.시제.unique()}\n\
{[len(train[train['시제']==train.시제.unique()[idx]]) for idx in range(len(train.시제.unique()))]}\n\
{train.확실성.unique()}\n\
{[len(train[train['확실성']==train.확실성.unique()[idx]]) for idx in range(len(train.확실성.unique()))]}\n"
)

1. num train : 16541

2. null train check : 
ID       0
문장       0
유형       0
극성       0
시제       0
확실성      0
label    0
dtype: int64

3. unique labels : 
['사실형' '추론형' '예측형' '대화형']
[13558, 2151, 257, 575]
['긍정' '부정' '미정']
[15793, 565, 183]
['현재' '과거' '미래']
[6866, 8032, 1643]
['확실' '불확실']
[15192, 1349]



In [6]:
def truncate(texts, data):
    truncated_texts = []
    for txt in texts:
        splited = txt.split(' ')
        if len(splited)>=40:
            valid = ' '.join(splited[:20] + splited[-20:])
            truncated_texts.append(valid)
        else:
            truncated_texts.append(txt)  
    data['문장'] = truncated_texts
    print('truncated')
    return

print(train['문장'][8838])
truncate(train['문장'], train)
print(train['문장'][8838])


반작(봄에 환곡을 나눠줄 때는 주지 않고도 주었다고 하고, 가을에 거두어들일 때는 회수하고도 회수하지 않았다고 해 중간에 이득을 나눔), 입본(농사 상황과 곡식 시세를 살펴서 돈과 곡식 간의 교환을 통해 이득을 챙김), 가집(상급 부서에서 지시한 것보다 더 많은 곡식을 방출하고 남는 것을 횡령), 암류(환곡을 제때에 대출하지 않고 창고에 쌓아 두었다가 값이 오르면 팔고 내리면 사들임), 반백(농민을 속여 대출 때 곡식의 절반을 가로채고 갚을 때는 모두 갚게 함), 분석(곡식에 돌, 쭉정이를 섞어 늘어난 양만큼 횡령), 집신(묵은 곡식은 나눠주고 햇곡식은 자기들이 가짐), 탄정(흉년이 들면 정부에서 환곡의 수량을 감해주는데 백성들에게는 환곡을 전량 징수하고 감액만큼 착복), 세전(환곡으로 받은 곡식과 세금으로 받은 곡식을 이리저리 돌려 이익을 남김), 요합(민간이 부역 대신 곡식으로 납부할 때 거슬러주어야 할 쌀을 횡령), 사흔(아전이 환곡을 징수하면서 자기들의 수고비를 같이 징수), 채륵(아전이 개인 채무까지 환곡과 혼합해 착복)이 그것이다.
truncated
반작(봄에 환곡을 나눠줄 때는 주지 않고도 주었다고 하고, 가을에 거두어들일 때는 회수하고도 회수하지 않았다고 해 중간에 이득을 나눔), 입본(농사 상황과 납부할 때 거슬러주어야 할 쌀을 횡령), 사흔(아전이 환곡을 징수하면서 자기들의 수고비를 같이 징수), 채륵(아전이 개인 채무까지 환곡과 혼합해 착복)이 그것이다.


## train data 둘러보기

In [ ]:
texts = train.문장.tolist()
len(texts)

# 문장 자르기
truncated_texts = []
for txt in texts:
    splited = txt.split(' ')
    if len(splited)>=40:
        valid = ' '.join(splited[:20] + splited[-20:])
        truncated_texts.append(valid)
    else:
        truncated_texts.append(txt)
truncated_texts[:3]

In [ ]:
lengths = []
for text in tq(texts):
    lengths.append(len(text.split(' ')))
lengths = pd.DataFrame(sorted(lengths))    
lengths[0].plot()

In [ ]:
lengths = []
for text in tq(truncated_texts):
    lengths.append(len(text.split(' ')))
lengths = pd.DataFrame(sorted(lengths))    
lengths[0].plot()

In [ ]:
encoding = t.encode_plus(
    truncated_texts[8838],
    add_special_tokens=True,
    max_length= 72, 
    return_token_type_ids=False,
    padding = 'max_length',
    truncation = True,
    return_attention_mask=True,
    return_tensors='pt'
)
encoding

In [ ]:
embeddings = AutoModel.from_pretrained("klue/roberta-large").embeddings.word_embeddings.weight
embeddings

In [ ]:
embeddings.shape

In [ ]:
temp.weight

In [ ]:
temp = torch.nn.Embedding(32000, 1024)
temp(torch.tensor([1,2,3]))

In [ ]:
embeddings(3)

### LSTM encoder

In [ ]:
class LSTMEncoder(torch.nn.Module):
    def __init__(self, I, E, H):
        super().__init__()
        self.embedding = AutoModel.from_pretrained("klue/roberta-large").embeddings.word_embeddings
        self.lstm = torch.nn.LSTM(input_size=E, hidden_size=H, num_layers=3, batch_first=True)
        
    def forward(self, x_ids:torch.tensor, h, c):
        x = self.embedding(x_ids)
        hiddens, (hidden, cell) = self.lstm(x, (hidden, cell))
        
        return hiddens, hidden, cell

In [ ]:
PLM.embeddings.word_embeddings.weight.shape

In [ ]:
PLM.embeddings.token_type_embeddings.weight

In [ ]:
encoding.tokens()

In [ ]:
print(texts[880])

In [ ]:
End_tagger = Komoran()
NVA_tagger = Okt()



In [ ]:
valid_tagger = Komoran()
noun_tagger = Okt()

In [ ]:
stopwords = set(["라며","전","이", "그", "저"])

In [ ]:
cur = texts[6]
print(cur)
cur = cur.split(' ')

valid_tokens = []
for token in cur:
    added_flag = False
    
    nouns  = noun_tagger.nouns(token)
    valids = valid_tagger.pos(token)
      
    for piece1, piece2 in zip_longest(nouns, valids):
        if piece1 is not None:
            if(piece1[0] not in stopwords):
                valid_tokens.append(piece1)
        if (piece2[1][0] == 'V') | (piece2[1][0] == 'X') | (piece2[1] == 'EP') | (piece2[1] == 'EF') | (piece2[1] == 'EC'):
            valid_tokens.append(piece2[0])
                        
print(valid_tokens, "\nlen : ",len(valid_tokens))

In [ ]:
    nouns  = noun_tagger.nouns(token)
    valids = valid_tagger.pos(token)

In [ ]:
noun_tagger.nouns(texts[5])

In [ ]:
NVA_tagger.pos(texts[1500])

In [ ]:
End_tagger.pos(texts[1800])

In [ ]:
NVA_tagger.pos('길어오기')

In [ ]:
End_tagger.pos('리뉴얼했고')

In [ ]:
Ends

In [ ]:
NVAs

In [ ]:
nouns = lambda x : (x[1] =='NNG') | (x[1] == 'NNP')
verbs = lambda x : x[1] == 'VV'
advs  = lambda x : x[1] == 'VA'
ends  = lambda x : (x[1] == 'EF') | (x[1] == 'EC')

valid = lambda x : ( nouns(x) | verbs(x) | advs(x) | ends(x) )


for elem in out:
    if (valid(elem)):
        print(elem)

## find frequent words

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("klue/roberta-large")
noun_ext = Okt()

In [ ]:
def wordcounter(data:pd.Series, mode='noun', threshold=30, visible_tqdm = True):
    dic = dict()
    noun_ext = Okt()

    if mode == 'noun':
        func = noun_ext.nouns
    elif mode == 'pos':
        func = noun_ext.pos
    else:
        func = noun_ext.morphs
    
    if visible_tqdm is not None:
        iterator = tq(enumerate(data), total=len(data))
    else:
        iterator = enumerate(data)
    

    for idx, text in iterator:
        nouns = func(text)

        for word in nouns:
            if len(word) <= 1 :
                continue 
            if word in dic:
                dic[word] += 1
            else:
                dic[word] = 1 

                    
    freq_all = sorted(dic.items(), key = lambda item: item[1], reverse = True)                
    freq = []
    
    # threshold 이하는 다 쳐내
    for item in freq_all:
        if item[1] < threshold:
            continue
        freq.append(item)
    
    freq = freq[:50]
    
    return freq

### frequent words of each "type" of text

##### (1) 사실형

In [ ]:
사실형 = train[train['유형']=='사실형']['문장'].tolist()
사실형 = wordcounter(사실형, mode=None, visible_tqdm=True)
사실형

##### (2) 추론형 

In [ ]:
추론형 = train[train['유형']=='추론형']['문장'].tolist()
추론형 = wordcounter(추론형, True)
추론형

##### (3) 예측형

In [ ]:
예측형 = train[train['유형']=='예측형']['문장'].tolist()
예측형 = wordcounter(예측형, True)
예측형

##### (4) 대화형

In [ ]:
대화형 = train[train['유형']=='대화형']['문장'].tolist()
대화형 = wordcounter(대화형, True)
대화형

### frequent words of each "polarity" of text

##### (1) 긍정

In [ ]:
긍정 = train[train['극성']=='긍정']['문장'].tolist()
긍정 = wordcounter(긍정, mode=None, visible_tqdm=True)
긍정

##### (2) 부정

In [ ]:
부정 = train[train['극성']=='부정']['문장'].tolist()
부정 = wordcounter(부정, mode=None, visible_tqdm=True)
부정


##### (3) 미정


In [ ]:
미정 = train[train['극성']=='미정']['문장'].tolist()
미정 = wordcounter(미정, mode=None, visible_tqdm=True)
미정


### frequent words of each "tense" of text

In [ ]:
현재 = train[train['시제']=='현재']['문장'].tolist()
현재 = wordcounter(현재, mode=None, visible_tqdm=True)
현재

In [ ]:
과거 = train[train['시제']=='과거']['문장'].tolist()
과거 = wordcounter(과거, mode=None, visible_tqdm=True)
과거

In [ ]:
미래 = train[train['시제']=='미래']['문장'].tolist()
미래 = wordcounter(미래, mode=None, visible_tqdm=True)
미래

### check test data
- weighted F1 score  
- 

In [ ]:
test.head(5)

### check submit data

In [ ]:
submit.head(5)

### pretrained model  
- 논문으로 공개된 사전 학습 모델(Pre-trained Model) 사용 가능  
- KLUE roberta-large

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, data, max_len, plm="klue/roberta-large", infer=False):
        self.text      = data['문장'].tolist()
        self.tokenizer = AutoTokenizer.from_pretrained(plm)
        self.max_len   = max_len
        self.infer     = infer
        
        if self.infer is not None:
            type1 = data['유형']=='사실형'
            type2 = data['유형']=='추론형'
            type3 = data['유형']=='예측형'
            type4 = data['유형']=='대화형'
            
            sent1 = data['극성']=='긍정'
            sent2 = data['극성']=='부정'
            sent3 = data['극성']=='미정'
            
            tense1 = data['시제']=='현재'
            tense2 = data['시제']=='과거'
            tense3 = data['시제']=='미래'
            
            certainty1 = data['확실성']=='확실'
            certainty2 = data['확실성']=='불확실'
            
            self.y1 = np.where(type1, 0, 
                      np.where(type2, 1, 
                      np.where(type3, 2, 3))).tolist()
            self.y2 = np.where(sent1, 0,
                      np.where(sent2, 1, 2)).tolist()
            self.y3 = np.where(tense1, 0, 
                      np.where(tense2, 1, 2)).tolist()
            self.y4 = np.where(certainty1, 0, 1).tolist()
                        
            
    def __len__(self):
        return len(self.text)
    
    
    def __getitem__(self, idx):
        text = self.text[idx]
        
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding = 'max_length',
            truncation = True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        
        x_ids, x_attn = [encoding['input_ids'].flatten(), encoding['attention_mask'].flatten()]
        if self.infer is not None:
            y1 = self.y1[idx]
            y2 = self.y2[idx]
            y3 = self.y3[idx]
            y4 = self.y4[idx]
            
            ys = torch.tensor([y1,y2,y3,y4]) 
            return x_ids, x_attn, ys        
        else:
            return x_ids, x_attn
            
def get_dataloader(data, batch_size, max_len=256, plm="klue/roberta-large", infer=False, shuffle=True):
    return DataLoader(
        dataset=CustomDataset(data=data, max_len=max_len, plm=plm, infer=infer),
        batch_size=batch_size,
        shuffle=shuffle
    )

### data loader

In [ ]:
class Classifier(torch.nn.Module):
    def __init__(self, plm = "klue/roberta-large"):
        super().__init__()
        self.activate = torch.nn.SiLU()
        self.dropout = torch.nn.Dropout(p=0.1)
        self.feature_extractor = AutoModel.from_pretrained(plm)
        self.feature_extractor.eval() # freeze FE 
    
        self.type_linear      = self.get_cls()
        self.polarity_linear  = self.get_cls()
        self.tense_linear     = self.get_cls()
        self.certainty_linear = self.get_cls()
            
    def forward(self, x_ids, x_attn):  
        
        x = self.feature_extractor(input_ids=x_ids, attention_mask=x_attn).to_tuple()[0] [:, 0, :] # cls token
                                                                                    # hidden states:0 / last hidden state:1
        y1 = self.type_linear(x)
        y2 = self.polarity_linear(x)
        y3 = self.tense_linear(x)
        y4 = self.certainty_linear(x)
    
        return (y1,y2,y3,y4)

    def get_cls(self):
        return torch.nn.Sequential(
            torch.nn.Linear(1024, 768),
            torch.nn.LayerNorm(768),
            torch.nn.Dropout(p=0.1),
            self.activate,
            torch.nn.Linear(768, 256),
            torch.nn.LayerNorm(256),
            torch.nn.Dropout(p=0.1),
            self.activate,
            torch.nn.Linear(256, 1)
        )  

In [ ]:
CFG = {
    'EPOCHS':10,
    'LEARNING_RATE':1e-4,
    'BATCH_SIZE':256,
    'SEED':41
}

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

In [ ]:
train_loader = get_dataloader(
    data=train,
    batch_size=CFG['BATCH_SIZE']
)

model = Classifier()
optimizer = torch.optim.Adam(model.parameters())

In [ ]:
criterions = [torch.nn.CrossEntropyLoss() for _ in range(4)]

In [ ]:
device = "cuda" if torch.cuda.is_available() else 'cpu'

model = model.to(device)
criterions = [torch.nn.CrossEntropyLoss().to(device) for _ in range(4)]

for epoch in range(1, CFG['EPOCHS']+1):
    model.train()
    train_loss=[]
    for x_ids, x_attn, y in tq(train_loader):
        x_ids, x_attn, y = x_ids.to(device), x_attn.to(device), y.to(device)
        
        optimizer.zero_grad()
        yhat0, yhat1, yhat2, yhat3 = model(x_ids, x_attn)
        
        loss1 = 0.25*criterions[0](yhat0, y[0])
        loss2 = 0.25*criterions[1](yhat1, y[1])
        loss3 = 0.25*criterions[2](yhat2, y[2])
        loss4 = 0.25*criterions[3](yhat3, y[3])
        
        loss = loss1+loss2+loss3+loss4
        
        loss.backward()
        optimizer.step()
        
        train_loss.append(loss.item())
        print(loss.item())